In [13]:
!pip install -U kaleido

In [14]:
pip install dash --quiet

Note: you may need to restart the kernel to use updated packages.


In [15]:
!pip install omnixai --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 1.30.1 requires protobuf<5,>=3.12.0, which is not installed.
tensorboard 2.10.0 requires protobuf<3.20,>=3.9.2, which is not installed.
tensorboardx 2.6.2.2 requires protobuf>=3.20, which is not installed.


In [16]:
!pip install dash-bootstrap-components --quiet
# После выполнения Перезапустить сеанс!

In [17]:
!pip install pycaret[full]

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.5 which is incompatible.


In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import math
import sklearn
import sklearn.ensemble


from omnixai.data.tabular import Tabular
from omnixai.preprocessing.base import Identity
from omnixai.preprocessing.tabular import TabularTransform
from omnixai.explainers.tabular import TabularExplainer
from omnixai.visualization.dashboard import Dashboard

ContextualVersionConflict: (pillow 10.4.0 (c:\users\render-pc\.conda\envs\pythonda2\lib\site-packages), Requirement.parse('pillow<10.0'), {'omnixai'})

Загрузка, подготовка данных, исследовательский анализ данных

In [ ]:
df = pd.read_csv('1223ia.csv', sep=';',decimal=',',encoding='utf-8-sig')

In [1]:
df.info()



NameError: name 'df' is not defined

In [ ]:
df['OrderDate'] = pd.to_datetime(df['OrderDate'],format='%d.%m.%Y')
df['ShipDate'] = pd.to_datetime(df['ShipDate'],format='%d.%m.%Y')

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.describe()

In [ ]:
df['Sales'] = df['UnitPrice'] * df['Quantity']

In [ ]:
df['SalesDiscount'] = df['Sales'] - df['DiscountAmount']

In [ ]:
df.head(3)

In [ ]:
(df.groupby('OrderDate').agg({'Sales': ['min', 'max'], 'SalesDiscount': 'sum'}).sort_values(by = 'OrderDate'))

In [ ]:
 df1 = df.groupby(pd.Grouper(key='OrderDate', axis=0, freq='M')).sum(numeric_only=True)

In [ ]:
df1.info()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df1.index, df1['Sales'], df1['SalesDiscount'], marker='o', linestyle='-')
plt.title('Продажи')
plt.xlabel('OrderDate')
plt.ylabel('Sales, SalesDiscount')
plt.grid(True)
plt.show()

In [ ]:
 df.groupby('ProductName').agg(SalesDiscount=('SalesDiscount', 'sum'), CustomerStateID=('CustomerStateID', 'first'), ProductID=('ProductID', 'first'), Channels=('Channels', 'first'),RetailPrice=('RetailPrice', 'first'),UnitPrice=('UnitPrice', 'first'),
  RegionID=('RegionID', 'first')).sort_values('SalesDiscount', ascending=False)

In [ ]:
df.groupby('ProductName').size().sort_values(ascending=True).plot(kind='barh', color=('blue'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
df.groupby('Demographic').size().sort_values(ascending=True).plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
df.groupby('Quantity').size().sort_values(ascending=True).plot(kind='barh', color=('#add8e9'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
sns.boxplot(x=df['UnitPrice'])

In [ ]:
sns.boxplot(x=df['SalesDiscount'])

In [ ]:
df.groupby('CustomerStateID').agg(SalesDiscount=('SalesDiscount', 'sum'), ProductID=('ProductID', 'first'), Channels=('Channels', 'first'),RetailPrice=('RetailPrice', 'first'),UnitPrice=('UnitPrice', 'first'),
  RegionID=('RegionID', 'first')).sort_values('SalesDiscount', ascending=False)

In [ ]:
df.groupby('RegionName').agg(SalesDiscount=('SalesDiscount', 'sum')) \
  .sort_values('SalesDiscount', ascending=False) \
  .head(3)

In [ ]:
df.groupby('StateName').agg(SalesDiscount=('SalesDiscount', 'sum')) \
  .sort_values('SalesDiscount', ascending=False) \
  .head(3)

Машинное обучение с библиотекой **PyCaret**

In [ ]:
df.columns

In [ ]:
data = df.drop(columns=['OrderNumber', 'OrderDate', 'Channels', 'ItemGroup',
      'KitType', 'Demographic',
       'ShipDate', 'ProductName', 'PromotionCode', 'StateName', 'RegionName',
       'ProductCategory', 'ProductSKU','StateCode'])

In [ ]:
data.corr(method='pearson')

In [2]:
data = df.drop(columns=['OrderNumber','UnitPrice','RetailPrice','Sales','ProductSKU','ProductID','ProductName','ShipDate','PromotionCode'])

NameError: name 'df' is not defined

In [ ]:
from pycaret.regression import *
s = setup(data, target='SalesDiscount', session_id=123, train_size=0.7, fold_shuffle=True, n_jobs=-1)

In [ ]:
# import RegressionExperiment and init the class
from pycaret.regression import RegressionExperiment
exp = RegressionExperiment()

In [ ]:
# check the type of exp
type(exp)

In [ ]:
exp = setup(data, target='SalesDiscount', session_id=123, train_size=0.7, fold_shuffle=True, n_jobs=-1)

In [ ]:
best_models = compare_models(fold=10, n_select=3, sort='RMSE')

In [ ]:
best = create_model('dt')

In [ ]:
plot_model(best, plot = 'residuals')

In [ ]:
# plot feature importance
plot_model(best, plot = 'feature')